## Cell 0 — Configure paths & load data

**Goal.** Load:
- `electrodes.csv` → names, costs, scalp coords.
- `usefulness_real.csv` → per-(subject, electrode) usefulness from baseline logistic.

**Notation.**
- \(E\): electrodes, \(S\): subjects.
- \(A=[a_{e,s}]\in[0,1]\): usefulness, \(c=[c_e]\): costs.
- We’ll minimize cost subject to a **usefulness target** and add a soft **symmetry** penalty.


In [ ]:
import pandas as pd, numpy as np
from pathlib import Path

ELEC_CSV = Path(r"C:\Users\0218s\Desktop\optimal-electrode-mip\data\electrodes.csv")
USE_CSV  = Path(r"C:\Users\0218s\Desktop\optimal-electrode-mip\data\usefulness_real.csv")

elec = pd.read_csv(ELEC_CSV)   # name,cost,location,x,y
use  = pd.read_csv(USE_CSV)    # subject,electrode,usefulness

E_list = sorted(elec["name"].astype(str).unique())
S_list = sorted(use["subject"].astype(str).unique())

# usefulness matrix a[e,s]
a_df = use.pivot_table(index="electrode", columns="subject", values="usefulness", aggfunc="mean")\
         .reindex(index=E_list, columns=S_list).fillna(0.0)

# costs vector
c_ser = elec.set_index("name")["cost"].reindex(E_list).fillna(1.0)

print("E =", len(E_list), "S =", len(S_list), "A-shape:", a_df.shape)


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\electrodes.csv'

## Cell 1 — Build left–right pairs \(P\) from coordinates

**Goal.** Encourage symmetric montages via soft penalty.
We automatically pair each left electrode with the nearest mirrored right electrode (exclude midline \(x\approx 0\)).


In [ ]:
XY = elec.set_index("name")[["x","y"]].astype(float)

left  = [e for e in E_list if XY.loc[e,"x"] < -1e-6]
right = [e for e in E_list if XY.loc[e,"x"] >  1e-6]

pairs = []
usedR = set()
for L in left:
    xL,yL = XY.loc[L,["x","y"]]
    # look for R near mirror of L across midline (target x ≈ +|xL|)
    cand = [(R, (XY.loc[R,"x"] - (-xL))**2 + (XY.loc[R,"y"] - yL)**2) for R in right if R not in usedR]
    if cand:
        R,_ = min(cand, key=lambda t:t[1])
        pairs.append((L,R))
        usedR.add(R)

P_list = [f"{L}|{R}" for (L,R) in pairs]
print("Pairs built:", len(P_list))
print("Sample pairs:", P_list[:10])


## Cell 2 — **Model B: Min-Cost with Usefulness Target** (ε-constraint) + Soft Symmetry

**Sets.** \(E\) electrodes, \(S\) subjects, \(P\) left–right pairs \(p=(\ell,r)\).

**Parameters.**
- \(a_{e,s}\in[0,1]\) usefulness, \(c_e>0\) cost.
- \(\alpha\in(0,1]\) target fraction of baseline usefulness.
- \(\lambda\ge 0\) symmetry penalty weight.

**Decision variables.**
- \(z_e\in\{0,1\}\) select electrode.
- \(s_p\ge 0\) asymmetry slack (measures \(|z_\ell - z_r|\)).

**Objective.**
\[
\min\ \sum_{e\in E} c_e z_e \;+\; \lambda \sum_{p\in P} s_p
\]

**Constraints.**
\[
\begin{aligned}
&\sum_{s\in S}\sum_{e\in E} a_{e,s} z_e \ \ge\ \tau_{\text{avg}} := \alpha \sum_{s\in S}\sum_{e\in E} a_{e,s} \quad \text{(usefulness target)}\\
& z_\ell - z_r \le s_p,\quad z_r - z_\ell \le s_p \quad \forall p=(\ell,r)\in P \\
& z_e\in\{0,1\}\ \forall e,\quad s_p\ge 0\ \forall p
\end{aligned}
\]

**Notes.**
- \(\alpha\) controls how close we must stay to baseline. Start at 0.90–0.95.
- Larger \(\lambda\) → more symmetry; \(\lambda=0\) ignores symmetry.


In [ ]:
import gamspy as gp
import gamspy.math as gpm

m = gp.Container()

E = gp.Set(m, "E", records=E_list)
S = gp.Set(m, "S", records=S_list)
P = gp.Set(m, "P", records=P_list)

# helpers to split "L|R"
def L_of(pid): return pid.split("|")[0]
def R_of(pid): return pid.split("|")[1]

# Parameters
A = gp.Parameter(m, "A", domain=[E,S])
C = gp.Parameter(m, "C", domain=[E])

A.setRecords(a_df.stack().reset_index().rename(columns={"electrode":"E","subject":"S",0:"A"}))
C.setRecords(pd.DataFrame({"E":E_list, "C":c_ser.values}))

# Decisions
z = gp.Variable(m, "z", domain=[E], type="binary")    # select electrode
s = gp.Variable(m, "s", domain=[P], type="positive")  # asymmetry slack

# Usefulness target τ_avg = α * baseline_total
baseline_total = float(a_df.values.sum())
alpha = 0.90         # <-- you can sweep this later
lam   = 0.5          # <-- symmetry penalty (try 0.1..1.0)
tau_avg_val = alpha * baseline_total

avg_target = gp.Equation(m, "avg_target")
avg_target[...] = gpm.Sum(S, gpm.Sum(E, A[E,S] * z[E])) >= tau_avg_val

# Symmetry constraints
for pid in P_list:
    L, R = L_of(pid), R_of(pid)
    gp.Equation(m, f"asym1_{L}_{R}", z[L] - z[R] <= s[pid])
    gp.Equation(m, f"asym2_{L}_{R}", z[R] - z[L] <= s[pid])

# Objective: Minimize total cost + λ * asymmetry
obj = gpm.Sum(E, C[E] * z[E]) + lam * gpm.Sum(P, s[P])

model = gp.Model(m, name="min_cost_target", sense=gp.Sense.MIN, problem=gp.Problem.MIP, objective=obj)
res = model.solve(solver="cplex")
print(res.status)


## Cell 3 — Read solution & save selection

**Outputs.**
- Selected electrodes (final montage).
- Total cost and total asymmetry slack.
- Save CSV for reproducibility.


In [ ]:
sol_z = z.toList()
chosen = [rec["E"] for rec in sol_z if rec["level"] > 0.5]
total_cost = float(sum(c_ser.loc[e] for e in chosen))
sym_sum = float(sum(rec["level"] for rec in s.toList()))

print(f"α = {alpha:.3f}  baseline_total = {baseline_total:.2f}")
print("Selected electrodes:", chosen)
print("Total cost:", total_cost)
print("Symmetry slack sum:", sym_sum)

Path(r"C:\Users\0218s\Desktop\optimal-electrode-mip\results").mkdir(parents=True, exist_ok=True)
pd.DataFrame({"electrode": chosen}).to_csv(
    r"C:\Users\0218s\Desktop\optimal-electrode-mip\results\selected_min_cost_alpha_{:.2f}.csv".format(alpha),
    index=False
)


## Cell 4 — α-sweep (Pareto curve: min cost vs required usefulness)

**Goal.** Trace the trade-off between **required usefulness** (α) and **minimum achievable cost** (with symmetry penalty).

**Outputs.** JSON with α, cost, symmetry, and selected set for each run.


In [ ]:
results = []
for alpha in [0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.85, 0.90, 0.95]:
    tau_avg_val = alpha * baseline_total
    m.getEquation("avg_target")[...] = gpm.Sum(S, gpm.Sum(E, A[E,S] * z[E])) >= tau_avg_val
    res = model.solve(solver="cplex")
    chosen = [rec["E"] for rec in z.toList() if rec["level"] > 0.5]
    cost   = float(sum(c_ser.loc[e] for e in chosen))
    sym    = float(sum(rec["level"] for rec in s.toList()))
    results.append({"alpha":alpha, "cost":cost, "symmetry":sym, "selected":chosen})

res_df = pd.DataFrame(results)
res_df.sort_values("alpha", inplace=True)
res_df.to_json(r"C:\Users\0218s\Desktop\optimal-electrode-mip\results\alpha_sweep.json",
               orient="records", indent=2)
res_df[["alpha","cost","symmetry"]]


## Cell 5 — Validation: Held-out ROC AUC for selected subsets

**Goal.** Confirm that subsets meeting the surrogate target also preserve **ROC AUC** on unseen data.

**Procedure.**
1. Load `features.csv`.
2. For each subset from the α-sweep, refit logistic on those electrodes with 5-fold CV.
3. Report mean ROC AUC vs cost (and α).


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

FEAT_CSV = Path(r"C:\Users\0218s\Desktop\optimal-electrode-mip\data\features.csv")
feat = pd.read_csv(FEAT_CSV)  # subject,electrode,feature,label

X_all = feat.pivot_table(index="subject", columns="electrode", values="feature", aggfunc="mean")\
           .reindex(columns=E_list).fillna(0.0)
y_all = feat.drop_duplicates("subject").set_index("subject")["label"].reindex(X_all.index).astype(int)

def auc5cv_for_subset(cols):
    if len(cols) == 0:
        return np.nan
    Xsub = X_all[cols].copy()
    # z-score per feature
    Xsub = (Xsub - Xsub.mean(0)) / (Xsub.std(0) + 1e-12)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    aucs = []
    for tr, va in skf.split(Xsub, y_all):
        clf = LogisticRegression(max_iter=300, solver="liblinear", class_weight="balanced")
        clf.fit(Xsub.iloc[tr], y_all.iloc[tr])
        p = clf.predict_proba(Xsub.iloc[va])[:,1]
        aucs.append(roc_auc_score(y_all.iloc[va], p))
    return float(np.mean(aucs))

res_df["auc5cv"] = res_df["selected"].apply(auc5cv_for_subset)
res_df[["alpha","cost","symmetry","auc5cv"]]
